## Text Encoding

In [42]:
with open("../data.txt", "r", encoding='utf-8') as f:
    text = f.read()

print(f"Length of dataset in characters: {len(text)}")

Length of dataset in characters: 1338608


In [43]:
## sample data
print(f"{text[:200]}")

Pro-Israel rallies allowed in India but Palestine solidarity sees crackdown
PM Hasina’s war on terror gets daughter India’s vote in WHO
India leads in migration to 'high-income' countries - report
Ind


In [3]:
## Get all the unique characters
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f"All unique characters: {''.join(chars)}")
print(f"Total number of characters: {vocab_size}")

All unique characters: 
 !"#$%&'()+,-./0123456789:;=?@ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz| £°Æàáçéíïñóúüġūʻ​–—‘’“”…€₹❤️
Total number of characters: 113


In [4]:
# Mapping char to int
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(f"Encoded : {encode('Hello there!')}")
print(f"Decoded: {decode(encode('Hello there!'))}")

Encoded : [38, 61, 68, 68, 71, 1, 76, 64, 61, 74, 61, 2]
Decoded: Hello there!


In [5]:
## convert thse into tensors
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(f"Shape: {data.shape}, Type: {data.dtype}")
print(f"Sample : {data[:200]}")

Shape: torch.Size([1338608]), Type: torch.int64
Sample : tensor([ 46,  74,  71,  13,  39,  75,  74,  57,  61,  68,   1,  74,  57,  68,
         68,  65,  61,  75,   1,  57,  68,  68,  71,  79,  61,  60,   1,  65,
         70,   1,  39,  70,  60,  65,  57,   1,  58,  77,  76,   1,  46,  57,
         68,  61,  75,  76,  65,  70,  61,   1,  75,  71,  68,  65,  60,  57,
         74,  65,  76,  81,   1,  75,  61,  61,  75,   1,  59,  74,  57,  59,
         67,  60,  71,  79,  70,   0,  46,  43,   1,  38,  57,  75,  65,  70,
         57, 105,  75,   1,  79,  57,  74,   1,  71,  70,   1,  76,  61,  74,
         74,  71,  74,   1,  63,  61,  76,  75,   1,  60,  57,  77,  63,  64,
         76,  61,  74,   1,  39,  70,  60,  65,  57, 105,  75,   1,  78,  71,
         76,  61,   1,  65,  70,   1,  53,  38,  45,   0,  39,  70,  60,  65,
         57,   1,  68,  61,  57,  60,  75,   1,  65,  70,   1,  69,  65,  63,
         74,  57,  76,  65,  71,  70,   1,  76,  71,   1,   8,  64,  65,  63,
       

In [6]:
## Split data into train and val set
n = int(0.9*len(data)) # 90% train, 10% val
train_data = data[:n]
val_data = data[n:]

In [7]:
## We are not going to put everything together, we use batches/chunks. 
block_size = 16
print(f"Chunk of size {block_size} represented by {train_data[:block_size+1]}")

Chunk of size 16 represented by tensor([46, 74, 71, 13, 39, 75, 74, 57, 61, 68,  1, 74, 57, 68, 68, 65, 61])


In [8]:
x = train_data[:block_size] # input data
y = train_data[1:block_size+1] # targets
for t in range(block_size): 
    context = x[:t+1]
    target = y[t]
    print(f"Input: {context} ---> Target: {target}")

Input: tensor([46]) ---> Target: 74
Input: tensor([46, 74]) ---> Target: 71
Input: tensor([46, 74, 71]) ---> Target: 13
Input: tensor([46, 74, 71, 13]) ---> Target: 39
Input: tensor([46, 74, 71, 13, 39]) ---> Target: 75
Input: tensor([46, 74, 71, 13, 39, 75]) ---> Target: 74
Input: tensor([46, 74, 71, 13, 39, 75, 74]) ---> Target: 57
Input: tensor([46, 74, 71, 13, 39, 75, 74, 57]) ---> Target: 61
Input: tensor([46, 74, 71, 13, 39, 75, 74, 57, 61]) ---> Target: 68
Input: tensor([46, 74, 71, 13, 39, 75, 74, 57, 61, 68]) ---> Target: 1
Input: tensor([46, 74, 71, 13, 39, 75, 74, 57, 61, 68,  1]) ---> Target: 74
Input: tensor([46, 74, 71, 13, 39, 75, 74, 57, 61, 68,  1, 74]) ---> Target: 57
Input: tensor([46, 74, 71, 13, 39, 75, 74, 57, 61, 68,  1, 74, 57]) ---> Target: 68
Input: tensor([46, 74, 71, 13, 39, 75, 74, 57, 61, 68,  1, 74, 57, 68]) ---> Target: 68
Input: tensor([46, 74, 71, 13, 39, 75, 74, 57, 61, 68,  1, 74, 57, 68, 68]) ---> Target: 65
Input: tensor([46, 74, 71, 13, 39, 75, 74

In [9]:
## Stacking inputs and target in matrix

torch.manual_seed(1337)
batch_size = 4  # num of seq to process in parallel
block_size = 8  # max context length

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print(f"Input: {xb}")
print(f"Input shape: {xb.shape}")

print(f"Target: {yb}")
print(f"Target shape: {yb.shape}")

print('-'*50)

## The ith element of target is the target of the ith element of input. It follows the same for rest.

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"Input: {context.tolist()} ---> Target: {target}")


Input: tensor([[76, 71,  1, 59, 71, 70, 76, 65],
        [ 1, 60, 61, 62, 74, 57, 77, 60],
        [61, 74,  1, 59, 71, 77, 70, 76],
        [57, 68,  1, 74, 57, 65, 70,  0]])
Input shape: torch.Size([4, 8])
Target: tensor([[71,  1, 59, 71, 70, 76, 65, 70],
        [60, 61, 62, 74, 57, 77, 60, 65],
        [74,  1, 59, 71, 77, 70, 76, 74],
        [68,  1, 74, 57, 65, 70,  0, 43]])
Target shape: torch.Size([4, 8])
--------------------------------------------------
Input: [76] ---> Target: 71
Input: [76, 71] ---> Target: 1
Input: [76, 71, 1] ---> Target: 59
Input: [76, 71, 1, 59] ---> Target: 71
Input: [76, 71, 1, 59, 71] ---> Target: 70
Input: [76, 71, 1, 59, 71, 70] ---> Target: 76
Input: [76, 71, 1, 59, 71, 70, 76] ---> Target: 65
Input: [76, 71, 1, 59, 71, 70, 76, 65] ---> Target: 70
Input: [1] ---> Target: 60
Input: [1, 60] ---> Target: 61
Input: [1, 60, 61] ---> Target: 62
Input: [1, 60, 61, 62] ---> Target: 74
Input: [1, 60, 61, 62, 74] ---> Target: 57
Input: [1, 60, 61, 62, 74, 

In [10]:
## INput to transformer
print(f"Input to the Transformer:\n{xb}")

Input to the Transformer:
tensor([[76, 71,  1, 59, 71, 70, 76, 65],
        [ 1, 60, 61, 62, 74, 57, 77, 60],
        [61, 74,  1, 59, 71, 77, 70, 76],
        [57, 68,  1, 74, 57, 65, 70,  0]])


## Building our Model

In [15]:
import torch 
import torch.nn as nn 
from torch.nn import functional as F 
import math
torch.manual_seed(1337)

n_embd = 32

class LanguageModel(nn.Module):
    
    def __init__(self, vocal_size):
        super().__init__()

        self.token_embedding_table = nn.Embedding(vocab_size, n_embd) # this will give the logits or the score of the next token

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None

        else:
            B, T, C = logits.shape 
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            # print(f"logits becomes of shape: {logits.view(B*T, C).shape}")
            # print(f"Targets becomes of shape: {targets.view(B*T).shape}")

            # to find how well are we predicting the next chars
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = LanguageModel(vocab_size) # vocab size = 3095
logits, loss = model(xb, yb)
print(f"Shape of logits: {logits.shape}")
print(f"Loss: {loss}")
print(f"Logit sample: {logits}")
print(f"The expected loss should be close to negative log likelihood: {-math.log(1/vocab_size)}")

idx = torch.zeros((1,1), dtype=torch.long)
max_new_tokens=100
print(decode(model.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


Shape of logits: torch.Size([256, 113])
Loss: 5.100133895874023
Logit sample: tensor([[ 0.0080,  0.9262, -1.8846,  ..., -0.3983, -0.5819, -0.2208],
        [ 2.0530, -1.0044,  1.2277,  ..., -0.0300, -0.7128, -0.7875],
        [ 0.3732, -1.0784, -1.0838,  ...,  0.4586, -0.8030, -0.7832],
        ...,
        [-0.2478, -1.4815, -0.0305,  ..., -0.0471,  1.2598,  0.5104],
        [-0.3446, -0.9640, -0.0594,  ..., -0.0032,  0.2197,  0.8017],
        [ 1.0753, -1.3365, -0.5952,  ...,  0.6615, -0.2995, -1.4161]],
       grad_fn=<ViewBackward0>)
The expected loss should be close to negative log likelihood: 4.727387818712341

1”xapo oNSm;Æ0f$SCT❤%XYk@£j?Rá️#í(%1áZ)!U+7ñ/:42z|éb;c+Fwm;Æ2yy1ZV️;‘O/–​i&hí?tD’K-LZk)WE=;zNuWD(”!|


In [40]:
# Pytorch optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

batch_size = 32
for steps in range(500): # increase number of steps for good results... 
    
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


2.7460267543792725


In [41]:
print(decode(model.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


Istseino Maicolomin peaulas a
K s Wetatherked at, potusre Kicighon AWel P ns ak
CConay
Chor fy arghi


## Transformers - Build Model

In [45]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x

tensor([[[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]],

        [[ 1.3488, -0.1396],
         [ 0.2858,  0.9651],
         [-2.0371,  0.4931],
         [ 1.4870,  0.5910],
         [ 0.1260, -1.5627],
         [-1.1601, -0.3348],
         [ 0.4478, -0.8016],
         [ 1.5236,  2.5086]],

        [[-0.6631, -0.2513],
         [ 1.0101,  0.1215],
         [ 0.1584,  1.1340],
         [-1.1539, -0.2984],
         [-0.5075, -0.9239],
         [ 0.5467, -1.4948],
         [-1.2057,  0.5718],
         [-0.5974, -0.6937]],

        [[ 1.6455, -0.8030],
         [ 1.3514, -0.2759],
         [-1.5108,  2.1048],
         [ 2.7630, -1.7465],
         [ 1.4516, -1.5103],
         [ 0.8212, -0.2115],
         [ 0.7789,  1.5333],
         [ 1.6097, -0.4032]]])

In [55]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
print(f"Lower triangular matrix:\n{wei}")
wei = wei / wei.sum(1, keepdim=True) # normalizing the weights along row
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C) # matrix multiplication of weights and input
print(f"weights of shape: {wei.shape} x X of shape: {x.shape}")
print(f"Output:\n{xbow2}")

Lower triangular matrix:
tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])
weights of shape: torch.Size([8, 8]) x X of shape: torch.Size([4, 8, 2])
Output:
tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]],

        [[-0.6631, -0.2513],
         [ 0.1735, -0.0649],
         [ 0.1685,  0.3348],
  

In [58]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
print(f"Lower triangular matrix:\n{tril}")
wei = torch.zeros((T,T))
print(f"zeroes matrix:\n{wei}")

wei = wei.masked_fill(tril == 0, float('-inf'))
print(f"New masked matrix:\n{wei}")
wei = F.softmax(tril, dim=-1)
print(f"Matrix after softmax:\n{wei}")
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

Lower triangular matrix:
tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])
zeroes matrix:
tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])
New masked matrix:
tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
       

False

## Self Attention layer

In [70]:
# version 4: self-attention!
import torch 
from torch import nn
from torch.nn import functional as F

torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

print(f"Batch B: {B}, Row T: {T}, Features C: {C}")

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
print(f"Linear heads:")
print(f"Key k: {key}, Query: {query}, Value: {value}")
print(f"Generating key, query and value from the features")


k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) * head_size**-0.5 # (B, T, 16) @ (B, 16, T) ---> (B, T, T) divided by sq root of heads size

print(f"Generating Key and Query from input x")
print(f"Matmul of query and Key.transpose of shapes {k.shape} and {q.shape} respectively we get weights:\n{wei.shape}")

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

print(f"Weights of shape: {wei.shape}")

v = value(x)

print(f"Generate value v from input of shape: {v.shape}")
out = wei @ v
#out = wei @ x
print(f"And we get the output of shape {out.shape} when wei @ x")

print(f"-"*50)

print("""
- scalling of softmax is used to control the variance at initialization
- Attention is a communication mechanism. It can be seen as a graph looking at each other. 
Our data has 8 nodes, an each node agreegates to the previous nodes. 1st node points to each self, 2nd points to 1st and itself, and it continues...
- This is something we call autoregressive
- These nodes have no idea where the other nodes are placed in the space, so we need positional encoding to let them know the positional information.
- Elements doesn't talk to each other across batches. Since we have 4 batches, they only talk inside the batch.
- If the all nodes talk to each other - it is encoder block
- If it doesn't - it is decoder
- In an "encoder" attention block just delete the single line that does masking with tril, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- In self attention, k, q, v comes from same space .... x-> k,q,v
- In cross attention, say q are prduced from x, but k, v are from different space, like from an encoder. Reading info from side, a separate source, to be pulled info from. 
    """)

Batch B: 4, Row T: 8, Features C: 32
Linear heads:
Key k: Linear(in_features=32, out_features=16, bias=False), Query: Linear(in_features=32, out_features=16, bias=False), Value: Linear(in_features=32, out_features=16, bias=False)
Generating key, query and value from the features
Generating Key and Query from input x
Matmul of query and Key.transpose of shapes torch.Size([4, 8, 16]) and torch.Size([4, 8, 16]) respectively we get weights:
torch.Size([4, 8, 8])
Weights of shape: torch.Size([4, 8, 8])
Generate value v from input of shape: torch.Size([4, 8, 16])
And we get the output of shape torch.Size([4, 8, 16]) when wei @ x
--------------------------------------------------

- scalling of softmax is used to control the variance at initialization
- Attention is a communication mechanism. It can be seen as a graph looking at each other. 
Our data has 8 nodes, an each node agreegates to the previous nodes. 1st node points to each self, 2nd points to 1st and itself, and it continues...
- Th

In [78]:
model

LanguageModel(
  (token_embedding_table): Embedding(113, 113)
)

In [76]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.tensor([encode("India demands")], dtype=torch.long)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


FileNotFoundError: [Errno 2] No such file or directory: 'input.txt'

In [75]:
context = torch.tensor([encode("India demands")], dtype=torch.long)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

NameError: name 'm' is not defined

In [74]:
torch.tensor([encode("India demands")], dtype=torch.long)

tensor([[39, 70, 60, 65, 57,  1, 60, 61, 69, 57, 70, 60, 75]])

In [72]:
torch.zeros((1, 1), dtype=torch.long)

tensor([[0]])